In [1]:
import pandas as pd

df_p_f = pd.read_csv('/kaggle/input/fakenewsnet/full_dataset/politifact_fake.csv')
df_p_r = pd.read_csv('/kaggle/input/fakenewsnet/full_dataset/politifact_real.csv')
df_g_f = pd.read_csv('/kaggle/input/fakenewsnet/full_dataset/gossipcop_fake.csv')
df_g_rs = [pd.read_csv(f'/kaggle/input/fakenewsnet/full_dataset/gossipcop_real_{i}.csv') for i in range(1,5)]
df_g_r = pd.concat(df_g_rs,ignore_index = True)

In [2]:
import numpy as np
def parse_url(url):
    if url != url:
        return np.nan
    id1 = url.find('//')
    if id1>=0:
        url = url[id1+2:]
    id2 = url.find('www.')
    if id2>=0:
        url = url[id2+4:]
    id3 = url.find('/')
    url = url[:id3]
    return url

In [3]:
df_p_f['parsed_url'] = df_p_f['news_url'].apply(lambda x: parse_url(x))
df_p_f['label'] = 0
df_p_r['parsed_url'] = df_p_r['news_url'].apply(lambda x: parse_url(x))
df_p_r['label'] = 1
df_g_f['parsed_url'] = df_g_f['news_url'].apply(lambda x: parse_url(x))
df_g_f['label'] = 0
df_g_r['parsed_url'] = df_g_r['news_url'].apply(lambda x: parse_url(x))
df_g_r['label'] = 1

In [4]:
everything_df = pd.concat([df_g_f,df_g_r,df_p_f,df_p_r],ignore_index = True)
everything_df.dropna(subset = ['parsed_url'], inplace=True)

In [5]:
set_true = list(set(everything_df[everything_df['label']==1]['parsed_url']))
set_false = list(set(everything_df[everything_df['label']==0]['parsed_url']))

In [6]:
to_classify = everything_df[(everything_df['parsed_url'].isin(set_true)) & (everything_df['parsed_url'].isin(set_false))]

In [7]:
! pip install happytransformer evaluate

from happytransformer import HappyTextClassification
happy_tc = HappyTextClassification("BERT", "bert-base-uncased", num_labels=2)

  Obtaining dependency information for happytransformer from https://files.pythonhosted.org/packages/f6/ed/8abe77d280294a454534003242431439b102cf17e1089fde6020f90ab621/happytransformer-3.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for datasets<3.0.0,>=2.13.1 from https://files.pythonhosted.org/packages/ec/93/454ada0d1b289a0f4a86ac88dbdeab54921becabac45da3da787d136628f/datasets-2.16.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec[http]>=2021.05.0 from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e71

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
import re
import nltk
import spacy
import string
from nltk.corpus import stopwords
nltk.download('wordnet')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
from nltk.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/co

In [9]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
PUNCT_TO_REMOVE = string.punctuation
STOPWORDS = set(stopwords.words('english'))
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

def preprocess (text):
    def lower(text):
        return text.lower()
    def remove_punctuation(text):
        return text.translate(text.maketrans('', '', PUNCT_TO_REMOVE))
    def remove_stopwords(text):
        return " ".join([word for word in str(text).split() if word not in STOPWORDS])
    def stem_words(text):
        return " ".join([stemmer.stem(word) for word in text.split()])
    def lemmatize_words(text):
        pos_tagged_text = nltk.pos_tag(text.split())
        return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
    return lemmatize_words(stem_words(remove_stopwords(remove_punctuation(lower(text)))))

In [27]:
to_classify_2 = to_classify[~to_classify['baseline'].isna()]
#to_classify_2['baseline_process'] = to_classify_2['baseline'].apply(lambda x: preprocess(x))

In [29]:
to_classify_2[to_classify_2['baseline'].isna()]

,id,news_url,title,tweet_ids,text,baseline,parsed_url,label


In [36]:
train_data = to_classify_2.sample(n=int(0.8*len(to_classify)))
test_data = to_classify_2[~to_classify_2.id.isin(train_data.id)]

In [37]:
train_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)

In [38]:
import csv
train_cases = train_data['baseline']
train_labels = train_data['label']

with open("/kaggle/working/train.csv", "w") as f:
   writer = csv.writer(f)
   writer.writerow(['text', 'label'])
   writer.writerows(zip(train_cases, train_labels))

In [40]:
from happytransformer import TCTrainArgs
args = TCTrainArgs(batch_size=8)
happy_tc.train("/kaggle/working/train.csv", args=args)

Generating train split: 0 examples [00:00, ? examples/s]

Tokenizing data:   0%|          | 0/12384 [00:00<?, ? examples/s]

Tokenizing data:   0%|          | 0/1376 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1,0.758700,0.655459
155,0.529000,0.456434
310,0.514300,0.497632
465,0.487700,0.504992
620,0.517500,0.467385
775,0.463700,0.459931
930,0.444500,0.464550
1085,0.418100,0.418509
1240,0.416500,0.427545
1395,0.399600,0.439754


In [44]:
from tqdm import tqdm
predictions = []

for case in tqdm(test_data['baseline']):
  output = happy_tc.classify_text(case[:512]).label
  if output == "LABEL_0":
    predictions.append(0)
  else:
    predictions.append(1)

  1%|          | 9/1377 [00:00<00:16, 83.76it/s]/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 1377/1377 [00:15<00:00, 87.14it/s]


In [45]:
import evaluate
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
test_labels = test_data['label']
accuracy_output = accuracy_metric.compute(references=test_labels, predictions=predictions)
print(accuracy_output)
precision__output = precision_metric.compute(references=test_labels, predictions=predictions)
print(precision__output)
recall_output = recall_metric.compute(references=test_labels, predictions=predictions)
print(recall_output)

{'accuracy': 0.8358750907770516}
{'precision': 0.8395061728395061}
{'recall': 0.9705042816365367}


In [47]:
2/(1/0.8395061728395061+1/0.9705042816365367)

0.9002647837599294